In [4]:
dp1 = {'parton' :100, 'maxtemp':35,'mintemp':35,'maxvibration':12,'asperity':0.32}
dp2 = {'parton' :101, 'maxtemp':46,'mintemp':35,'maxvibration':21,'asperity':0.34}
dp3 = {'parton' :130, 'maxtemp':56,'mintemp':46,'maxvibration':3412,'asperity':12.42}
dp4 = {'parton' :131, 'maxtemp':58,'mintemp':48,'maxvibration':3542,'asperity':13.43}

In [2]:
dp

{'parton': 100,
 'maxtemp': 35,
 'mintemp': 35,
 'maxvibration': 12,
 'asperity': 0.32}

In [4]:
!git clone https://github.com/saeedahmadian/sample_data.git

Cloning into 'sample_data'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [6]:
! ls sample_data

Data.csv  README.md


In [8]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('Regression').getOrcreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load('sample_data/Data.csv')


In [10]:
type(df_data)

pyspark.sql.dataframe.DataFrame

In [11]:
df_data.show(10)

+--------+----+-------+------+------+-----+-----+------+------+-----+-----+------+-------+------+-------+-----+
|    Date|Hour|DA_DEMD|DEMAND|DA_LMP|DA_EC|DA_CC|DA_MLC|RT_LMP|RT_EC|RT_CC|RT_MLC|DryBulb|DewPnt|SYSLoad|RegCP|
+--------+----+-------+------+------+-----+-----+------+------+-----+-----+------+-------+------+-------+-----+
|1/1/2011|   1|11722.0| 12055| 38.51|38.37|  0.0|  0.14| 41.57|41.48|  0.0|  0.09|     36|    29|  12214| 6.24|
|1/1/2011|   2|11253.0| 11430| 36.86|36.75|  0.0|  0.11| 34.91|34.82|  0.0|  0.09|     36|    29|  11573| 9.18|
|1/1/2011|   3|10459.0| 10966| 35.92|35.91|  0.0|  0.01| 35.46|35.29|  0.0|  0.17|     35|    29|  11114| 7.15|
|1/1/2011|   4|10531.0| 10725| 35.94|35.91|  0.0|  0.03| 34.14|33.97|  0.0|  0.17|     35|    30|  10863|  7.4|
|1/1/2011|   5|10841.0| 10672| 36.05|36.04|  0.0|  0.01| 33.63|33.49|  0.0|  0.14|     36|    31|  10827| 9.69|
|1/1/2011|   6|10661.0| 10852| 36.05|36.12|  0.0| -0.07| 34.25|34.11|  0.0|  0.14|     36|    31|  11005

In [32]:
df_data.printSchema

<bound method DataFrame.printSchema of DataFrame[Date: string, Hour: int, DA_DEMD: double, DEMAND: int, DA_LMP: double, DA_EC: double, DA_CC: double, DA_MLC: double, RT_LMP: double, RT_EC: double, RT_CC: double, RT_MLC: double, DryBulb: int, DewPnt: int, SYSLoad: int, RegCP: double]>

In [ ]:
from pyspark.ml.regression import 

In [37]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

assemble_inp = VectorAssembler(inputCols=['DA_DEMD','DA_LMP','DA_EC','DA_MLC','DryBulb','DewPnt','SYSLoad','RegCP'],outputCol='features')
assemble_out = VectorAssembler(inputCols=['RT_LMP','RT_MLC'],outputCol='target')
norm1 = Normalizer(inputCol='features',outputCol='norm_features')
norm2 = Normalizer(inputCol='RT_LMP',outputCol='norm_target')
data = Pipeline(stages=[assemble_inp,norm1]).fit(df_data).transform(df_data)

data.show()


+--------+----+-------+------+------+-----+-----+------+------+-----+-----+------+-------+------+-------+-----+--------------------+--------------------+
|    Date|Hour|DA_DEMD|DEMAND|DA_LMP|DA_EC|DA_CC|DA_MLC|RT_LMP|RT_EC|RT_CC|RT_MLC|DryBulb|DewPnt|SYSLoad|RegCP|            features|       norm_features|
+--------+----+-------+------+------+-----+-----+------+------+-----+-----+------+-------+------+-------+-----+--------------------+--------------------+
|1/1/2011|   1|11722.0| 12055| 38.51|38.37|  0.0|  0.14| 41.57|41.48|  0.0|  0.09|     36|    29|  12214| 6.24|[11722.0,38.51,38...|[0.69241987523854...|
|1/1/2011|   2|11253.0| 11430| 36.86|36.75|  0.0|  0.11| 34.91|34.82|  0.0|  0.09|     36|    29|  11573| 9.18|[11253.0,36.86,36...|[0.69711868277195...|
|1/1/2011|   3|10459.0| 10966| 35.92|35.91|  0.0|  0.01| 35.46|35.29|  0.0|  0.17|     35|    29|  11114| 7.15|[10459.0,35.92,35...|[0.68531486620140...|
|1/1/2011|   4|10531.0| 10725| 35.94|35.91|  0.0|  0.03| 34.14|33.97|  0.0| 

In [38]:
reg = LinearRegression(featuresCol='norm_features',labelCol='RT_LMP',predictionCol='predicted_values')
model =reg.fit(data)
summary = model.summary

In [41]:
summary.pValues

[0.007407330700269776,
 0.19047403545522323,
 6.73908484571939e-10,
 4.320543922631259e-12,
 4.440892098500626e-15,
 1.8784795365878182e-05,
 0.0,
 0.0031886893286663387,
 1.185238573953029e-10]

In [58]:
summary.r2

0.4695565911543459

In [59]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
# a = np.arange(0,10)
# summary.residuals.count()
# plt.scatter(np.arange(summary.residuals.count()),summary.residuals)


In [53]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
!git clone https://github.com/wchill/HMP_Dataset.git

from pyspark.sql.types import StructType,StructField,IntegerType

schema = StructType([
    StructField("x",IntegerType(),True),
    StructField("y",IntegerType(),True),
    StructField("z",IntegerType(),True),
])
import os
files=os.listdir('HMP_Dataset')
file_filter =[s for s in files if '_' in s]
from pyspark.sql.functions import lit
df = None
for category in file_filter:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    for data in data_files:
        print(data)
        tmp = spark.read.option('header','false').option('delimiter',' ').csv('HMP_Dataset/'+category+'/'+data,schema=schema)
        
        tmp = tmp.withColumn('class',lit(category))
        tmp = tmp.withColumn('source',lit(data))
        
        if df == None:
            df = tmp
        else:
            df = df.union(tmp)
df.show()

Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 902.00 KiB/s, done.
Checking out files: 100% (848/848), done.
Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt
Accelerometer-2011-03-24-10-24-39-climb_stairs-f1.txt
Accelerometer-2011-03-24-10-25-44-climb_stairs-f1.txt
Accelerome

### now I want to temparary convert spark datafram to spark sql so that I can run SQL codes for filtering However I can use dataframe.filter methods as well


In [14]:
df.createOrReplaceTempView('df')
df_energy = spark.sql("""

select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class

""")

df_energy.show(20)

+------------------+--------------+
|             label|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [16]:
type(df_energy)

pyspark.sql.dataframe.DataFrame

In [17]:
df_energy.createOrReplaceTempView('df_energy')
df_join = spark.sql("""

select * from df inner join df_energy on df.class = df_energy.class

""")
df_join.show(20)

+---+---+---+-----------+--------------------+-----------------+-----------+
|  x|  y|  z|      class|              source|            label|      class|
+---+---+---+-----------+--------------------+-----------------+-----------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|

In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

assembler = VectorAssembler(inputCols=['x','y','z'],outputCol='features')
norm = Normalizer(inputCol='features',outputCol='norm_feature')

In [21]:
from pyspark.ml.regression import LinearRegression
hyper_par = LinearRegression(maxIter=10,regParam=0.0,elasticNetParam=0.0)

In [22]:
from pyspark.ml import Pipeline
model_pip = Pipeline(stages=[assembler,norm,hyper_par])
data= model_pip.fit(df_join).transform(df_join)

In [26]:
data.show()

+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|      class|              source|            label|      class|        features|        norm_feature|        prediction|
+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.34316403829308...| 11102.77591357913|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.34316403829308...| 11102.77591357913|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.33117360613218...| 11122.75750783091|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.33117360613218...| 11122.75750783091|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|11785.3963446292

In [30]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")



'Pipeline_403aaf007c5b1327ebc1'